In [3]:
###################################### ESTRATEGIA BB
###################################################################################################################
import modulos as md
import numpy as np
import constantes as cons
import os
import sys

lista_filtrada = []
#lista=md.lista_de_monedas ()
#lista=md.filtradodemonedas ()
lista=['LINKUSDT',
 'LTCUSDT',
 'BCHUSDT',
 'WLDUSDT',
 'IDUSDT',
 'HBARUSDT',
 'API3USDT',
 'BNBUSDT',
 'C98USDT',
 '1000SHIBUSDT',
 'STXUSDT',
 'LPTUSDT',
 'XLMUSDT',
 'CRVUSDT',
 'MKRUSDT',
 'ALGOUSDT',
 'HIGHUSDT',
 'COMPUSDT',
 'DOGEUSDT',
 'XRPUSDT',
 'OPUSDT',
 'ARBUSDT',
 'AUDIOUSDT',
 '1000PEPEUSDT',
 'MATICUSDT',
 'APEUSDT',
 'SOLUSDT',
 'TOMOUSDT',
 'EOSUSDT',
 'ADAUSDT',
 'SUIUSDT',
 'YGGUSDT',
 'DODOXUSDT']

#lista=['AUDIOUSDT']

for symbol in lista:    
    try:
        data,porcentajeentrada = md.estrategia_santa(symbol,tp_flag = True)
        #data = md.estrategia_triangulos(symbol, tp_flag = True, print_lines_flag = True)
        #######################################################################
        #resultado = md.backtesting(data, plot_flag = True)
        resultado = md.backtestingsanta(data, plot_flag = False)
        if resultado['Return [%]'] >= 0:# and resultado['# Trades']!=0 :
            #if ((resultado['Profit Factor'] > 2 or np.isnan(resultado['Profit Factor'])) and (resultado['Return [%]']/resultado['# Trades']) >=0.33):            
                lista_filtrada.append(symbol)
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {resultado['Profit Factor']} - Win Rate [%]: {resultado['Win Rate [%]']}")
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+"\n")
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)
#md.dibuja_patrones_triangulos (data,998)    


AUDIOUSDT - Return [%]: -10.17% - # Trades: 14 - Profit Factor: 0.0996444702559628 - Win Rate [%]: 35.714285714285715


In [ ]:
data,porcentajeentrada = md.estrategia_santa(symbol,tp_flag = True)
        #data = md.estrategia_triangulos(symbol, tp_flag = True, print_lines_flag = True)
        #######################################################################
        #resultado = md.backtesting(data, plot_flag = True)
        resultado = md.backtestingsanta(data, plot_flag = False)
        if resultado['Return [%]'] >= 0:# and resultado['# Trades']!=0 :
            #if ((resultado['Profit Factor'] > 2 or np.isnan(resultado['Profit Factor'])) and (resultado['Return [%]']/resultado['# Trades']) >=0.33):            
                lista_filtrada.append(symbol)
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {resultado['Profit Factor']} - Win Rate [%]: {resultado['Win Rate [%]']}")
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+"\n")
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)
#md.dibuja_patrones_triangulos (data,998)    


In [2]:
data,porcentajeentrada = md.estrategia_santa(symbol,tp_flag = True)

UnboundLocalError: local variable 'data' referenced before assignment

In [ ]:
md.dibuja_patrones_triangulos (data,998) 